In [ ]:
# Set up the database
import mysql.connector

try:
    db = mysql.connector.connect(
        host="localhost",
        port=int(3306),
        user="root",
        passwd="Jaisurya7@",
        db="events_info"
    )

    cr = db.cursor()# Define the cursor object
    db.commit()
    
    
    

    # Define functions for CRUD operations
    
    def add_event(name, date, time, location, attendees, agenda,available_tickets):
        cr.execute("INSERT INTO events (name, date, time, location, attendees, agenda, available_tickets) VALUES (%s, %s, %s, %s, %s, %s, %s)", (name, date, time, location, attendees, agenda, available_tickets))
        db.commit()

    def delete_event(id):
        cr.execute("DELETE FROM events WHERE id=%s", (id,))
        db.commit()

    def update_event(id, name, date, time, location, attendees, agenda,available_tickets):
        cr.execute("UPDATE events SET name=%s, date=%s, time=%s, location=%s, attendees=%s, agenda=%s, available_tickets=%s WHERE id=%s", (name, date, time, location, attendees, agenda,available_tickets, id))
        db.commit()

    def view_events():
        cr.execute("SELECT * FROM events")
        events = cr.fetchall()
        for event in events:
            print(event)

    def generate_report():
        cr.execute("SELECT name, attendees, FROM events")
        report = cr.fetchall()
        for row in report:
            print(row[0] + ": " + str(row[1]) + " attendees")

    # Define a new function to register a user and return their user_id
    def register_user():
        username = input("Enter a username: ")
        password = input("Enter a password: ")

        cr.execute("SELECT COUNT(*) FROM users WHERE username = %s", (username,))
        result = cr.fetchone()
        if result[0] > 0:
            print("User already exists.")
            return None

        cr.execute("INSERT INTO users (username, password, role) VALUES (%s, %s, %s)", (username, password, "student"))
        db.commit()
        print("User registered successfully.")

        cr.execute("SELECT id FROM users WHERE username = %s", (username,))
        user_id = cr.fetchone()[0]
        return user_id
    
    

    # Define a new function to book a ticket for a user and event
    def book_ticket(user_id, event_id):
        cr.execute("SELECT available_tickets FROM events WHERE id=%s", (event_id,))
        result = cr.fetchone()

        if result is None:
            print("Event does not exist.")
            return False
        available_tickets = result[0]
        if available_tickets <= 0:
            print("No more available tickets.")
            return False
        cr.execute("UPDATE events SET available_tickets = available_tickets - 1 WHERE id=%s", (event_id,))
        cr.execute("INSERT INTO bookings (user_id, event_id) VALUES (%s, %s)", (user_id, event_id))
        db.commit()
        return True
    

   
   
        
# Define CLI using python code


    while True:
        
        
        role = input("Enter your role (Admin/User/Register): ")
    

        if role.lower() == "admin":
            admin_password = input("Enter the admin password: ")
            cr.execute("SELECT password FROM pswd WHERE username = 'admin'")
            result = cr.fetchone()

            if result is None:
                print("Admin account does not exist.")
            else:
                password = result[0]
                if admin_password == password:
                    print("Admin password is successful.")
                    action = input("Enter action (Add/Delete/Update): ")

                    if action.lower() == "add":
                        name = input("Enter event name: ")
                        date = input("Enter event date (YYYY-MM-DD): ")
                        time = input("Enter event time (HH:MM AM/PM): ")
                        location = input("Enter event location: ")
                        attendees = input("Enter number of attendees: ")
                        agenda = input("Enter event agenda: ")
                        available_tickets=input("enter the no of tickets: ")
                        add_event(name, date, time, location, attendees, agenda,available_tickets)
                        print("Event added successfully!")
                        view_events()

                    elif action.lower() == "delete":
                        id = input("Enter event ID: ")
                        delete_event(id)
                        print("Event deleted successfully")
                        view_events()

                    elif action.lower() == "update":
                        id = input("Enter event ID: ")
                        name = input("Enter event name: ")
                        date = input("Enter event date (YYYY-MM-DD): ")
                        time = input("Enter event time (HH:MM AM/PM): ")
                        location = input("Enter event location: ")
                        attendees = input("Enter number of attendees: ")
                        agenda = input("Enter event agenda: ")
                        available_tickets=input("enter the no of tickets that you want to update: ")
                        update_event(id, name, date, time, location, attendees, agenda,available_tickets)
                        print("Event updated successfully!")
                        view_events()
    
                    else:
                        print("Invalid action. Try again.")

                else:
                    print("Invalid admin password.")
                    

        elif role.lower() == "user":
            username = input("Enter your username: ")
            password = input("Enter your password: ")
            cr.execute("SELECT usernames, passwords FROM users WHERE rolee = 'student'")
            results = cr.fetchall()
            
            
            
            for result in results:
                
                username_db, password_db = result
                
                    
                user_id=0
                if username == username_db and password==password_db:
                    print("user login sucessful: ")
                    view_events()
                    
                    event_id = input("Enter the ID of the event you want to book: ")
                    if book_ticket( user_id,event_id):
                        print("Event booked successfully")
                    else:
                        print("Booking failed")
                    view_events()
                    
                    report = input("Do you want to generate a report? (Y/N): ")
                    if report.lower() == "y":
                        generate_report()
                    elif report.lower() == "n":
                        break
                    else:
                        print("Invalid input. Try again.")
                    break
            else:
                print("invalid username or password")


        elif role.lower() == "register":
            register_user()

        else:
            print("Invalid role. Try again.") #now continue statement is working
            continue

            
            

except Exception as e:
    db.close()
    print(str(e))
    

    